In [ ]:
#### make sure you install pyspark
!pip install -U pyspark --quiet

In [1]:
### importing libraeis
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession

In [2]:
# creating spark session

spark = (
    SparkSession.builder
        .master("local[*]")
        .appName("Experiment 5")
        .config("spark.driver.memory", "1g")
        .config("spark.executer", "3")
        .config("spark.executer.memory", "1g")
        .getOrCreate()
       )

24/09/25 10:10:13 WARN Utils: Your hostname, nooman resolves to a loopback address: 127.0.1.1; using 192.168.15.39 instead (on interface enp4s0)
24/09/25 10:10:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/25 10:10:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
spark.read.csv("/Ecommerce/Customers/Customers.csv", inferSchema=True, header=True).schema

StructType([StructField('CustomerID', IntegerType(), True), StructField('FirstName', StringType(), True), StructField('LastName', StringType(), True), StructField('Date_of_Birth', TimestampType(), True), StructField('City', StringType(), True), StructField('State', StringType(), True), StructField('Country', StringType(), True), StructField('PostalCode', IntegerType(), True), StructField('Phone', LongType(), True), StructField('Email', StringType(), True), StructField('DateEntered', TimestampType(), True)])

In [5]:
from pyspark.sql.types import StructField, StringType, IntegerType, StringType, DateType, StructType

customers_schema = StructType(
    [
        StructField('CustomerID', IntegerType(), True),
        StructField('FirstName', StringType(), True),
        StructField('LastName', StringType(), True),
        StructField('Date_of_Birth', DateType(), True),
        StructField('City', StringType(), True),
        StructField('State', StringType(), True),
        StructField('Country', StringType(), True),
        StructField('PostalCode', StringType(), True),
        StructField('Phone', StringType(), True),
        StructField('Email', StringType(), True),
        StructField('DateEntered', DateType(), True)
    ]
)

orders_schema = """
OrderID INT,
CustomerID INT,
PaymentID INT,
OrderDate DATE,
ShipperID INT,
ShipDate Date,
DeliveryDate Date, 
Total_order_amount float
"""

In [12]:
# I am reading loading from hdfs if you don't have hadoop in your system
# download the datasets usign this link 
## https://drive.google.com/drive/folders/1VzW2eBDCjOc4zzG-dSObFLyQxBSWVbn2
customers = spark.read.csv("/Ecommerce/Customers/Customers.csv", schema=customers_schema, header=True)
orders = spark.read.csv("/Ecommerce/Orders/Orders.csv", schema=orders_schema, header=True)
payments = spark.read.csv("/Ecommerce/Payments/Payments.csv", inferSchema=True, header=True)
products = spark.read.csv("/Ecommerce/Products/Products.csv", inferSchema=True, header=True)
orderdetails = spark.read.csv("/Ecommerce/OrderDetails/OrderDetails.csv", inferSchema=True, header=True)

In [6]:
#### Create a temporary table which is volatile when saprk session is close it will gone if you will 
# create global temporary it will store into Hive metastore

customers.createOrReplaceTempView("Customers")
orders.createOrReplaceTempView("Orders")
payments.createOrReplaceTempView("Payments")
products.createOrReplaceTempView("Products")
orderdetails.createOrReplaceTempView("OrderDetails")

In [7]:
customers.show(2, truncate=False)

+----------+---------+---------+-------------+--------+--------+-------------+----------+----------+--------------------------+-----------+
|CustomerID|FirstName|LastName |Date_of_Birth|City    |State   |Country      |PostalCode|Phone     |Email                     |DateEntered|
+----------+---------+---------+-------------+--------+--------+-------------+----------+----------+--------------------------+-----------+
|57081     |James    |Smith    |1987-03-26   |New York|New York|United States|280862    |9638483934|James.Smith@gmail.com     |2020-01-02 |
|57082     |Robert   |Downey Jr|1973-05-24   |New York|New York|United States|376573    |6588282115|Robert.Downey Jr@gmail.com|2020-01-06 |
+----------+---------+---------+-------------+--------+--------+-------------+----------+----------+--------------------------+-----------+
only showing top 2 rows



In [8]:
orders.show(2, truncate=False)

+-------+----------+---------+----------+---------+----------+------------+------------------+
|OrderID|CustomerID|PaymentID|OrderDate |ShipperID|ShipDate  |DeliveryDate|Total_order_amount|
+-------+----------+---------+----------+---------+----------+------------+------------------+
|7655500|57083     |2        |2020-01-12|7        |2020-01-13|2020-01-19  |25112.0           |
|7655501|57086     |3        |2020-01-20|2        |2020-01-24|2020-01-27  |22453.0           |
+-------+----------+---------+----------+---------+----------+------------+------------------+
only showing top 2 rows



In [9]:
products.show(2, truncate=False)

+---------+-------------------------------------------+-----------+--------------------+------+----------+------------+-----------------------------+
|ProductID|Product                                    |Category_ID|Sub_Category        |Brand |Sale_Price|Market_Price|Type                         |
+---------+-------------------------------------------+-----------+--------------------+------+----------+------------+-----------------------------+
|1        |Original Disinfectant Toilet Cleaner Liquid|5001       |All Purpose Cleaners|Harpic|489       |534         |Toilet Cleaners              |
|2        |Surface Disinfectant Spray                 |5001       |All Purpose Cleaners|Savlon|257       |318         |Disinfectant Spray & Cleaners|
+---------+-------------------------------------------+-----------+--------------------+------+----------+------------+-----------------------------+
only showing top 2 rows



In [10]:
payments.show(2)

+---------+-----------+-------+
|PaymentID|PaymentType|Allowed|
+---------+-----------+-------+
|        1| Debit Card|    Yes|
|        2|        POD|    Yes|
+---------+-----------+-------+
only showing top 2 rows



In [11]:
products.show(2)

+---------+--------------------+-----------+--------------------+------+----------+------------+--------------------+
|ProductID|             Product|Category_ID|        Sub_Category| Brand|Sale_Price|Market_Price|                Type|
+---------+--------------------+-----------+--------------------+------+----------+------------+--------------------+
|        1|Original Disinfec...|       5001|All Purpose Cleaners|Harpic|       489|         534|     Toilet Cleaners|
|        2|Surface Disinfect...|       5001|All Purpose Cleaners|Savlon|       257|         318|Disinfectant Spra...|
+---------+--------------------+-----------+--------------------+------+----------+------------+--------------------+
only showing top 2 rows



In [12]:
orderdetails.show(2)

+-------------+-------+---------+--------+----------+
|OrderDetailID|OrderID|ProductID|Quantity|SupplierID|
+-------------+-------+---------+--------+----------+
|            1|7655500|    14955|       2|         3|
|            2|7655500|    19946|       4|         2|
+-------------+-------+---------+--------+----------+
only showing top 2 rows



##### Find the Monthly revenue and %change in revenue with previous month
##### Find the product which is given 75% revneue of total
##### Find the year monthly  wise revenue and commulative revenue
##### Find the avg differnce between two consicative orders and divide into 3 bucket
##### Most selling products in terms of quantity in each country
##### check is there any trend every month some different product on peak
##### avg deleivery time for each country rank them in descending orders

In [12]:
#Find the Monthly revenue and %change in revenue with previous month

query = """
WITH CTE AS
(
    SELECT 
        YEAR(OrderDate) as Years,
        MONTH(OrderDate) as Months,
        CAST(SUM(Total_order_amount) AS INT) as Revenue
    FROM Orders
    GROUP BY YEAR(OrderDate), MONTH(OrderDate)
)
SELECT *, 
    LAG(Revenue, 1) OVER(PARTITION BY Years ORDER BY Months) AS Previous_month_revenue,
    ROUND(((Revenue - LAG(Revenue, 1) OVER(PARTITION BY Years ORDER BY Months))/Revenue)*100, 2) AS per_change
FROM CTE
"""
spark.sql(query).show()

+-----+------+-------+----------------------+----------+
|Years|Months|Revenue|Previous_month_revenue|per_change|
+-----+------+-------+----------------------+----------+
| 2020|     1|  47565|                  NULL|      NULL|
| 2020|     2| 200109|                 47565|     76.23|
| 2020|     3| 318971|                200109|     37.26|
| 2020|     4| 561036|                318971|     43.15|
| 2020|     5| 891360|                561036|     37.06|
| 2020|     6|1196040|                891360|     25.47|
| 2020|     7|1061605|               1196040|    -12.66|
| 2020|     8|1218623|               1061605|     12.88|
| 2020|     9|1678720|               1218623|     27.41|
| 2020|    10|1792856|               1678720|      6.37|
| 2020|    11|2051862|               1792856|     12.62|
| 2020|    12|2208324|               2051862|      7.09|
| 2021|     1|2696530|                  NULL|      NULL|
| 2021|     2|2611481|               2696530|     -3.26|
| 2021|     3|4211055|         

In [13]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col
from pyspark.sql.window import Window
yoy_sales = (
orders.groupBy(
    F.year(F.col('orderdate')).alias("Years"),
    F.month(F.col('OrderDate')).alias("Months")
)
.agg(F.sum(col("Total_order_amount")).cast("int").alias("Revenue"))
)

windowSpec = Window.partitionBy("Years").orderBy("Months")

yoy_sales.withColumns(
    {"previos_moth_sale" : F.lag(col("Revenue"), 1).over(windowSpec),
    "percent_channge": F.round((col("Revenue") - F.lag(col("Revenue"), 1).over(windowSpec))/col("Revenue") * 100, 2)}
).show()

+-----+------+-------+-----------------+---------------+
|Years|Months|Revenue|previos_moth_sale|percent_channge|
+-----+------+-------+-----------------+---------------+
| 2020|     1|  47565|             NULL|           NULL|
| 2020|     2| 200109|            47565|          76.23|
| 2020|     3| 318971|           200109|          37.26|
| 2020|     4| 561036|           318971|          43.15|
| 2020|     5| 891360|           561036|          37.06|
| 2020|     6|1196040|           891360|          25.47|
| 2020|     7|1061605|          1196040|         -12.66|
| 2020|     8|1218623|          1061605|          12.88|
| 2020|     9|1678720|          1218623|          27.41|
| 2020|    10|1792856|          1678720|           6.37|
| 2020|    11|2051862|          1792856|          12.62|
| 2020|    12|2208324|          2051862|           7.09|
| 2021|     1|2696530|             NULL|           NULL|
| 2021|     2|2611481|          2696530|          -3.26|
| 2021|     3|4211055|         

In [14]:
### Find the year monthly wise revenue and commulative revenue
import seaborn as sns
query = """

WITH CTE AS
(
SELECT 
    YEAR(OrderDate) AS Years,
    MONTH(OrderDate) AS Months,
    CAST(SUM(Total_order_amount) AS INT) AS Revenue
FROM Orders
GROUP BY YEAR(OrderDate), MONTH(OrderDate)
)
SELECT *,
    SUM(Revenue) OVER(PARTITION BY Years ORDER BY Months) AS Commulative_Revenue
FROM CTE
"""
spark.sql(query).show()

+-----+------+-------+-------------------+
|Years|Months|Revenue|Commulative_Revenue|
+-----+------+-------+-------------------+
| 2020|     1|  47565|              47565|
| 2020|     2| 200109|             247674|
| 2020|     3| 318971|             566645|
| 2020|     4| 561036|            1127681|
| 2020|     5| 891360|            2019041|
| 2020|     6|1196040|            3215081|
| 2020|     7|1061605|            4276686|
| 2020|     8|1218623|            5495309|
| 2020|     9|1678720|            7174029|
| 2020|    10|1792856|            8966885|
| 2020|    11|2051862|           11018747|
| 2020|    12|2208324|           13227071|
| 2021|     1|2696530|            2696530|
| 2021|     2|2611481|            5308011|
| 2021|     3|4211055|            9519066|
| 2021|     4|4404256|           13923322|
| 2021|     5|5081762|           19005084|
| 2021|     6|5215665|           24220749|
| 2021|     7|7014047|           31234796|
| 2021|     8|6911452|           38146248|
+-----+----

Find the top product which is given 10% revneue of total

In [15]:
query = """
WITH CTE1 AS
(
SELECT 
    c.ProductID, 
    c.Product,
    SUM(a.Total_order_amount) AS Revenue,
    SUM(a.Total_order_amount)/(SELECT SUM(Total_order_amount) FROM Orders) * 100 AS percent_revenue
FROM Orders AS a
JOIN OrderDetails AS b
ON a.OrderID = b.OrderID
JOIN Products AS c
ON b.ProductID = c.ProductID
GROUP BY c.ProductID, c.Product
),
CTE2 AS
(
SELECT *,
    SUM(percent_revenue) OVER(ORDER BY Revenue DESC) AS commulative_percent
FROM CTE1
)
SELECT 
    ProductID,
    Product,
    Revenue
FROM CTE2
WHERE commulative_percent <= 10
"""

spark.sql(query).show()

24/09/20 09:53:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/20 09:53:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/20 09:53:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/20 09:53:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/20 09:53:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/20 09:53:08 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/09/20 0

+---------+--------------------+------------------+
|ProductID|             Product|           Revenue|
+---------+--------------------+------------------+
|    12576|Raw Manuka Honey ...|         309849.75|
|    18730|Futura Hard Anodi...|          282753.0|
|     4711|AQVA Divina Body ...|         275639.75|
|     4638|Be Delicious Eau ...|    271510.8515625|
|    19320|Lily Ville Square...|          257149.0|
|    20295|Quantum Max Dishw...|     247793.953125|
|     4139|New Imperial 1000...|          236096.0|
|    12290|Chunky Orange Mar...|   234557.80078125|
|     6927|Hot Water Rubber ...|234380.35009765625|
|     4817|Voyage Sport Eau ...|     233160.078125|
|    14645|       Bhringraj Oil| 231098.7998046875|
|    15164|Healthy Khakhra -...|          229811.0|
|    18568|Stainless Steel C...|          223229.0|
|    20326|   Laundry Detergent|    220203.4765625|
|    10939|Liver Tonic & App...|   220038.69921875|
|      833|Baby Bottle & Foo...|          216971.0|
|    10759|A

Find the avg differnce between two consicative orders and divide into 3 bucket


In [38]:
from pyspark.sql.window import Window
customer_order = (
customers.join(
          orders,
          on="CustomerID")
        .select("CustomerID", "FirstName", "LastName", "OrderID", "OrderDate")
)

windowspec = Window.partitionBy(F.col("CustomerID")).orderBy(F.col("OrderDate"))
difference = customer_order.withColumns(
    {
     "previous_order": F.lag(F.col("OrderDate")).over(windowspec),
     "difference": F.date_diff(F.col("OrderDate"), F.lag(F.col("OrderDate")).over(windowspec))
    }
)
difference.groupBy(
    F.col("CustomerID"), 
    F.col("FirstName"), 
    F.col("LastName")).agg(
    F.round(F.avg(F.col("difference")), 2).alias("avg_difference")
).orderBy(F.col("avg_difference")).filter(F.col("avg_difference").isNotNull()).show()

+----------+---------+--------+--------------+
|CustomerID|FirstName|LastName|avg_difference|
+----------+---------+--------+--------------+
|     57605|  Malcolm|  Julian|           0.0|
|     57604|    Jesse| Francis|           1.0|
|     57603|    Idris|   Colby|          1.14|
|     57594|    River|   Louie|           1.5|
|     57597|   Szymon|   Asher|          1.62|
|     57601|  Darragh|   Niall|           1.7|
|     57600|    Bruce|  Lennox|          1.73|
|     57598|      Tom|  Bailey|           1.8|
|     57602|     Dean|Alistair|          1.83|
|     57596|  Stanley|     Ali|           2.0|
|     57593|  Oliwier|   Dylan|          2.25|
|     57592|    Munro|   Blair|           2.5|
|     57599|    Tymon|    Enzo|           3.0|
|     57595|    Ruari| Vincent|          3.57|
|     57589|    Marco|   Arran|          3.75|
|     57586|      Kit|  Hamish|           4.0|
|     57583|   Conall|   David|           4.1|
|     57585|     Ivor|    Ryan|          4.22|
|     57574| 

In [41]:
query = """
WITH CTE AS
(
SELECT 
    a.CustomerID,
    FirstName,
    LastName,
    OrderID,
    OrderDate,
    LAG(OrderDate, 1) OVER(PARTITION BY a.CustomerID ORDER BY OrderDate) AS previous_orderdate,
    DATEDIFF(OrderDate, LAG(OrderDate, 1) OVER(PARTITION BY a.CustomerID ORDER BY OrderDate)) AS difference
FROM Customers AS a
JOIN Orders AS b
ON a.CustomerID = b.CustomerID
)
SELECT 
    CustomerID,
    FirstName,
    LastName,
    ROUND(AVG(difference), 2) as avg_difference
FROM CTE
GROUP BY CustomerID, FirstName, LastName
HAVING ROUND(AVG(difference), 2) IS NOT NULL
ORDER BY avg_difference
"""

spark.sql(query).show()

+----------+---------+--------+--------------+
|CustomerID|FirstName|LastName|avg_difference|
+----------+---------+--------+--------------+
|     57605|  Malcolm|  Julian|           0.0|
|     57604|    Jesse| Francis|           1.0|
|     57603|    Idris|   Colby|          1.14|
|     57594|    River|   Louie|           1.5|
|     57597|   Szymon|   Asher|          1.62|
|     57601|  Darragh|   Niall|           1.7|
|     57600|    Bruce|  Lennox|          1.73|
|     57598|      Tom|  Bailey|           1.8|
|     57602|     Dean|Alistair|          1.83|
|     57596|  Stanley|     Ali|           2.0|
|     57593|  Oliwier|   Dylan|          2.25|
|     57592|    Munro|   Blair|           2.5|
|     57599|    Tymon|    Enzo|           3.0|
|     57595|    Ruari| Vincent|          3.57|
|     57589|    Marco|   Arran|          3.75|
|     57586|      Kit|  Hamish|           4.0|
|     57583|   Conall|   David|           4.1|
|     57585|     Ivor|    Ryan|          4.22|
|     57574| 

##### Most selling products in terms of quantity in each country

In [54]:
windowspec = Window.partitionBy(F.col("Country")).orderBy(F.col("total_quantity").desc())
(
customers.join(orders, 
               on="CustomerID")
         .join(orderdetails, on="OrderID")
         .join(products, on="ProductID")
         .select("Country", "ProductID", "Product","Quantity")
         .groupBy("Country", "ProductID", "Product").agg(
             F.sum(F.col("Quantity")).alias("total_quantity")
         ).withColumn("rank", F.dense_rank().over(windowspec))
         .filter(F.col("rank") == 1)
).show()

+----------------+---------+--------------------+--------------+----+
|         Country|ProductID|             Product|total_quantity|rank|
+----------------+---------+--------------------+--------------+----+
|       Australia|    21319|BBPopular Almond/...|            35|   1|
|         Austria|    13902|    Skin Therapy Oil|            33|   1|
|         Austria|    11912|           Pav Bhaji|            33|   1|
|         Belgium|    14069|Honey & Rose Body...|            39|   1|
|          France|     2184|Milk Shakti Biscu...|            42|   1|
|         Germany|    12058|Rice - Ponni, Boiled|            37|   1|
|          Greece|    15203|Kochi Masala Bana...|            53|   1|
|           India|     3947|Liquid Food Colou...|            57|   1|
|         Ireland|    17531|Wooden Printed Ir...|            39|   1|
|           Italy|     1237|Glycerin Germ Pro...|            35|   1|
|     Netherlands|      510|Alkaline Battery ...|            36|   1|
|     New Zealand|  

#### check is there any trend every month some different product on peak

In [63]:
window_spec = Window.partitionBy(F.col("months")).orderBy(F.col("quantity_sold").desc())
(
    orders.join(orderdetails, on="OrderID")
    .join(products, on="ProductID")
    .groupby(F.month(F.col("OrderDate")).alias("months"), 
             F.col("ProductID"),
             F.col("Product"))
    .agg(
        F.sum(F.col("Quantity")).alias("quantity_sold")
    )
    .withColumn("rank", F.dense_rank().over(window_spec))
    .filter(F.col("rank") == 1)
).show()

+------+---------+--------------------+-------------+----+
|months|ProductID|             Product|quantity_sold|rank|
+------+---------+--------------------+-------------+----+
|     1|    10481|Party Eye Mask - ...|           30|   1|
|     2|      638|Orthodontic Sooth...|           34|   1|
|     3|     6195|Taft Classic Hair...|           34|   1|
|     3|    16921|Copper Steel Wate...|           34|   1|
|     4|    17623|Square Pet water ...|           37|   1|
|     5|    20009|Moong Dal/Hesaru ...|           39|   1|
|     6|     6962|     Hot Water - Bag|           38|   1|
|     7|    19599|        Pepper Gouda|           38|   1|
|     8|     7965|Kashmiri Chilly P...|           40|   1|
|     9|     3558|Dark Chocolate - ...|           45|   1|
|    10|    20538|Ultra Soft Premiu...|           48|   1|
|    11|    16108|Peanut Butter - C...|           49|   1|
|    11|     9422|Cleaning Brush - ...|           49|   1|
|    12|     6226|NHP 8103 Silky Sh...|           51|   

##### avg deleivery time for each country rank them in descending orders¶

In [73]:
windowspec = Window.partitionBy("Country").orderBy(F.col("avg_lead_time"))
(
 customers.join(orders, on="CustomerID")
 .groupBy("Country", "City").agg(
     F.avg(F.date_diff(F.col("DeliveryDate"), F.col("OrderDate"))).alias("avg_lead_time"))
     .withColumn("rank", F.row_number().over(windowspec))
).show()

+---------+--------------+------------------+----+
|  Country|          City|     avg_lead_time|rank|
+---------+--------------+------------------+----+
|Australia|Sunshine Coast|10.833333333333334|   1|
|Australia|     Newcastle|              11.5|   2|
|Australia|      Canberra|12.333333333333334|   3|
|Australia|       Geelong|13.142857142857142|   4|
|Australia|        Cairns|14.210526315789474|   5|
|Australia|      Brisbane|15.419354838709678|   6|
|Australia| Central Coast|              15.5|   7|
|Australia|      Ballarat| 15.76923076923077|   8|
|Australia|     Melbourne|15.833333333333334|   9|
|Australia|         Perth|              15.9|  10|
|Australia|    Townsville| 16.42105263157895|  11|
|Australia|        Darwin|16.454545454545453|  12|
|Australia|        Hobart|17.863636363636363|  13|
|Australia|      Adelaide| 18.41176470588235|  14|
|Australia|    Wollongong|              18.6|  15|
|Australia|        Sydney|             22.25|  16|
|  Austria|     Innsbruck|12.55